## Generation of MUBs for 2 and 3 qubits

In [ ]:
from qiskit.opflow import X, Y, Z, I, PauliOp, PauliSumOp
from qsymm.linalg import simult_diag
import numpy as np
from bqskit.compiler import Compiler
from bqskit.compiler import CompilationTask

In [2]:
import os

MUB_CIRC_2_PATH = os.path.join(os.getcwd(), 'mub_bqskit', '2_302')
MUB_CIRC_3_PATH = os.path.join(os.getcwd(), 'mub_bqskit', '3_306')

The method in this section of the notebook is based on the paper "Mutually unbiased binary observable sets on N qubits"
By Lawrence, Brukner and Zeilinger. (2022)

The paper provides a table. In that table, each row contains several Pauli strings.
The basis that this row represents is the eigenvector basis that diagonalizes **the entire row**.

Thus, this function applies simultaneous diagonalization to all matrices (Pauli strings) from each row, to get the MUB.
After that, BQSKit is used to synthesize a circuit that generates those MUB states.

In [3]:
def mats_to_mub_circ(mats_row: np.ndarray, nqubits: int):
    mub_uni = np.hstack(simult_diag(mats_row))
    task = CompilationTask.synthesize(mub_uni)
    with Compiler() as compiler:
        synth_qc = compiler.compile(task)
    return mub_uni, synth_qc.to('qasm')

In [ ]:
tbl_2_302 = [ [Z^I, I^Z, Z^Z],
        [X^I, I^Y, X^Y],
        [Y^I, I^X, Y^X],
        [Y^Y, Z^X, Z^X],
        [X^X, Y^Z, Z^Y]]

tbl_3_306 = [   # The full columns were not added, as to save space, since 3 matrices should define the basis completely.
    [X^I^I, I^Y^I, I^I^Z, X^Y^Z, X^Y^I, X^I^Z, I^Y^Z],  # First 3 rows are product state bases
    [Y^I^I, I^Z^I, I^I^X],
    [Z^I^I, I^X^I, I^I^Y],
    [Y^Z^Z, Z^Y^Z, Z^Z^Y],  # Last 6 rows are GHZ-like bases
    [Z^X^X, X^Z^X, X^X^Z],
    [X^Y^Y, Y^X^Y, Y^Y^X],
    [Z^X^Z, Y^X^X, Y^Y^Z],
    [X^Y^X, Z^Y^Y, Z^Z^X],
    [Y^Z^Y, X^Z^Z, X^X^Y]
]



qasm_2_302 = {}
qasm_3_306 = {}

print('----------RESULTS FOR 2-QUBIT MUBS (3,0,2)--------------')
for i, row in enumerate(tbl_2_302):
    res = mats_to_mub_circ(list(map(lambda p: p.to_matrix(), row)), 2)
    print(f'result for row {i+1}:')
    print(res[0])
    print(res[1])
    with open(os.path.join(MUB_CIRC_2_PATH, str(i+1)+'.txt'), 'w') as f:
        f.write(res[1])
    qasm_2_302[i+1] = res[1]
    print('\n')


print('----------RESULTS FOR 3-QUBIT MUBS (3,0,6)--------------')
for i, row in enumerate(tbl_3_306):
    res = mats_to_mub_circ(list(map(lambda p: p.to_matrix(), row)), 3)
    print(f'result for row {i+1}:')
    print(res[0])
    print(res[1])
    with open(os.path.join(MUB_CIRC_3_PATH, str(i+1) + '.txt'), 'w') as f:
        f.write(res[1])
    qasm_3_306[i+1] = res[1]
    print('\n')